In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms
import sys

sys.path.insert(1, '../../software')
from models import *


In [2]:
#send an input and grap the value by using prehook like HW3


In [3]:
weight_int2 = torch.randint(low=-8, high=8, size=torch.Size([8,8,3,3])).float() # w_int can be calculated by weight_q and w_delta
#weight_int2 = (7 * torch.ones(torch.Size([8,8,3,3]))).float() # w_int can be calculated by weight_q and w_delta
#print(weight_int) # you should see clean integer numbers

In [4]:
x_int2 = torch.randint(low=0,high=16, size=torch.Size([128,8,4,4])).float()
#x_int2 = (15 * torch.ones(torch.Size([128,8,4,4]))).float()
#print(x_int) # you should see clean integer numbers 

In [5]:
conv_int2 = torch.nn.Conv2d(in_channels = 8, out_channels=16, kernel_size = 3, padding=1, bias = False)
conv_int2.weight = torch.nn.parameter.Parameter(weight_int2)

output_int2 =  conv_int2.forward(x_int2)    # output_int can be calculated with conv_int and x_int
relu = torch.nn.ReLU()
relu_output_int2 = relu(output_int2)
#print(output_recovered)

In [6]:


# act_int.size = torch.Size([128, 64, 32, 32])  <- batch_size, input_ch, ni, nj
a_int = x_int2[0,:,:,:]  # pick only one input out of batch
# a_int.size() = [64, 32, 32]

# conv_int.weight.size() = torch.Size([64, 64, 3, 3])  <- output_ch, input_ch, ki, kj
w_int = torch.reshape(weight_int2, (weight_int2.size(0), weight_int2.size(1), -1))  # merge ki, kj index to kij
# w_int.weight.size() = torch.Size([64, 64, 9])
                      
padding = 1
stride = 1
array_size = 8 # row and column number

nig = range(a_int.size(1))  ## ni group
njg = range(a_int.size(2))  ## nj group

icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel

ic_tileg = range(int(len(icg)/array_size))
oc_tileg = range(int(len(ocg)/array_size))

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

######## Padding before Convolution #######
a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(nig)+padding*2).cuda()
# a_pad.size() = [64, 32+2pad, 32+2pad]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))
# a_pad.size() = [64, (32+2pad)*(32+2pad)]


a_tile = torch.zeros(len(ic_tileg), array_size,    a_pad.size(1)).cuda() 
w_tile = torch.zeros(len(oc_tileg)*len(ic_tileg), array_size, array_size, len(kijg)).cuda() 

for ic_tile in ic_tileg:
    a_tile[ic_tile,:,:] = a_pad[ic_tile*array_size:(ic_tile+1)*array_size,:]

for ic_tile in ic_tileg:
    for oc_tile in oc_tileg:
        w_tile[oc_tile*len(oc_tileg) + ic_tile,:,:,:] = w_int[oc_tile*array_size:(oc_tile+1)*array_size, ic_tile*array_size:(ic_tile+1)*array_size, :]



###########################################

p_nijg = range(a_pad.size(1)) ## psum nij group

psum = torch.zeros(len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)).cuda() 

for kij in kijg:
    for ic_tile in ic_tileg:       # Tiling into array_sizeXarray_size array
        for oc_tile in oc_tileg:   # Tiling into array_sizeXarray_size array        
            for nij in p_nijg:       # time domain, sequentially given input
                    m = nn.Linear(array_size, array_size, bias=False)
                    #m.weight = torch.nn.Parameter(w_int[oc_tile*array_size:(oc_tile+1)*array_size, ic_tile*array_size:(ic_tile+1)*array_size, kij])
                    m.weight = torch.nn.Parameter(w_tile[len(oc_tileg)*oc_tile+ic_tile,:,:,kij])
                    psum[ic_tile, oc_tile, :, nij, kij] = m(a_tile[ic_tile,:,nij]).cuda()
 

In [7]:
import math

a_pad_ni_dim = int(math.sqrt(a_pad.size(1))) # 32

o_ni_dim = int((a_pad_ni_dim - (ki_dim- 1) - 1)/stride + 1)
o_nijg = range(o_ni_dim**2)    

print(len(o_nijg))

out = torch.zeros(len(ocg), len(o_nijg)).cuda()
  
   
### SFP accumulation ###
for o_nij in o_nijg: 
    for kij in kijg:
        for ic_tile in ic_tileg:    
            for oc_tile in oc_tileg:   
                out[oc_tile*array_size:(oc_tile+1)*array_size, o_nij] = out[oc_tile*array_size:(oc_tile+1)*array_size, o_nij] + \
                psum[ic_tile, oc_tile, :, int(o_nij/o_ni_dim)*a_pad_ni_dim + o_nij%o_ni_dim + int(kij/ki_dim)*a_pad_ni_dim + kij%ki_dim, kij]
                ## 4th index = (int(o_nij/30)*32 + o_nij%30) + (int(kij/3)*32 + kij%3)

16


In [8]:
### show this cell partially. The following cells should be printed by students ###
tile_id = 0 
nij = 200 # just a random number
X = a_tile[tile_id,:,:]  # [tile_num, array row num, time_steps]

bit_precision = 4
file = open('activation.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(round(X[7-j,i].item()))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file    


In [9]:
### Complete this cell ###
tile_id = 0 
kij = 0


bit_precision = 4
for kij in range(9):
    W = w_tile[tile_id,:,:,kij]  # w_tile[tile_num, array col num, array row num, kij]
    file = open('weight_' + str(kij) + '.txt', 'w') #write to file
    file.write('#col7row0[msb-lsb],col6row0[msb-lst],....,col0row0[msb-lst]#\n')
    file.write('#col7row1[msb-lsb],col6row1[msb-lst],....,col0row1[msb-lst]#\n')
    file.write('#................#\n')
    
    for i in range(W.size(1)):  # row
        for j in range(W.size(0)): # col #
            W_bin = '{0:04b}'.format(round(W[W.size(0)-1-j,i].item()) + (2**bit_precision if (round(W[W.size(0)-1-j,i].item()) < 0) else 0))
            for k in range(bit_precision):
                file.write(W_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
    file.close() #close file   

In [10]:
### Complete this cell ###
ic_tile_id = 0 
oc_tile_id = 0 


kij = 0
nij = 200
# psum[len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)]


bit_precision = 16

for kij in range(9):
    psum_tile = psum[ic_tile_id,oc_tile_id,:,:,kij]  
    file = open('psum_' + str(kij) + '.txt', 'w') #write to file
    file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
    file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
    file.write('#................#\n')

    for i in range(psum_tile.size(1)):  # time step
        for j in range(psum_tile.size(0)): # col #
            #psum_bin = '{0:016b}'.format(round(psum_tile[psum_tile.size(0)-1-j,i].item()) + (2**bit_precision if (round(psum_tile[psum_tile.size(0)-1-j,i].item()) < 0) else 0))
            curr_psum = round(psum_tile[psum_tile.size(0)-1-j,i].item())
            if (i == 7 and kij == 0):
                print(curr_psum)
            if (curr_psum < 0):
                if (i == 7 and kij == 0):
                    print('{0:016b}'.format(curr_psum))
                curr_psum = curr_psum + 2**bit_precision
                if (i == 7 and kij == 0):
                    print('{0:016b}'.format(curr_psum))
            psum_bin = '{0:016b}'.format(curr_psum)
            
            for k in range(bit_precision):
                file.write(psum_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
    file.close() #close file 

40
-277
-000000100010101
1111111011101011
-70
-000000001000110
1111111110111010
130
-52
-000000000110100
1111111111001100
-107
-000000001101011
1111111110010101
39
-31
-000000000011111
1111111111100001


In [11]:
### Complete this cell ###


bit_precision = 16
# out is array of size columns x len(o_nijg)

file = open('out.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(out.size(1)):  # time step
    for j in range(out.size(0)): # row #
        out_bin = '{0:016b}'.format(0 if (round(out[out.size(0)-1-j,i].item()) < 0) else (round(out[out.size(0)-1-j,i].item())))
        for k in range(bit_precision):
            file.write(out_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file 

In [12]:
### show this cell partially. The following cells should be printed by students ###
tile_id = 0 
nij = 200 # just a random number
X = a_tile[tile_id,:,:]  # [tile_num, array row num, time_steps]
print(X.size())

bit_precision = 4
file = open('activation_os.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

test_acc_vec = []

# the nature of convolution on output stationary makes it difficult to avoid data redundancy in the activation stream
# TODO: wrap in another loop for output tile
for ky in range(ki_dim):
    for kx in range(ki_dim):
        printed = False
        for j in range(X.size(0)): # input channels (column)
            for it in reversed(range(2)): # output channel row tile (row tile 0-1)
                for i in reversed(range(4)):  # time step (row number 0-3)
                    # if not printed:
                    #     print(ky*a_pad_ni_dim + kx + it*a_pad_ni_dim + i, '\t', end='')
                    if it == 0 and i == 1:
                        test_acc_vec.append(round(X[j,ky*a_pad_ni_dim + kx + it*a_pad_ni_dim + i].item()))
                        print(round(X[j,ky*a_pad_ni_dim + kx + it*a_pad_ni_dim + i].item()), end= ' ')
                    X_bin = '{0:04b}'.format(round(X[j,ky*a_pad_ni_dim + kx + it*a_pad_ni_dim + i].item()))
                    for k in range(bit_precision):
                        file.write(X_bin[k])        
                    #file.write(' ')  # for visibility with blank between words, you can use
            file.write('\n')
            printed = True
        # print()
file.close() #close file    

torch.Size([8, 36])
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 10 11 2 4 1 7 14 5 6 14 5 0 13 5 13 10 9 9 11 6 4 7 1 3 11 10 12 14 11 4 1 5 12 15 2 8 14 2 0 0 4 0 13 9 3 3 3 3 

In [13]:
print(w_tile[0,:,:,0])

tensor([[ 0., -2., -4.,  5.,  0.,  6., -7.,  7.],
        [-5.,  1.,  6., -6.,  1.,  0.,  6.,  1.],
        [ 0., -8.,  1., -7., -4.,  4., -3.,  5.],
        [-6.,  1.,  3., -7.,  2.,  2.,  2., -5.],
        [-4.,  7.,  3., -6.,  0.,  1.,  6.,  4.],
        [-7., -8., -4.,  4.,  6.,  1.,  3.,  5.],
        [-6., -5., -8.,  1.,  2., -7., -7., -1.],
        [ 7., -1.,  7., -8., -8., -8.,  7., -7.]], device='cuda:0')


In [14]:
print(a_tile[0,:,:])

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  6.,  9.,  6.,  0.,  0., 11.,
         12.,  4., 12.,  0.,  0.,  3., 13.,  3.,  6.,  0.,  0., 10.,  8.,  3.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 11., 14.,  9.,  7.,  0.,  0., 10.,
         15.,  0.,  6.,  0.,  0.,  9., 12.,  3., 15.,  0.,  0.,  2., 11.,  6.,
         11.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  5., 11.,  1.,  0.,  0., 12.,
          2., 13.,  9.,  0.,  0.,  5.,  2.,  2.,  3.,  0.,  0.,  8.,  8.,  2.,
          6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,  6.,  8.,  0.,  0., 14.,
          8.,  9.,  7.,  0.,  0.,  5.,  7.,  8., 15.,  0.,  0., 14.,  2.,  7.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., 13.,  4., 13.,  0.,  0., 11.,
         14.,  3.,  1.,  0.,  0.,  3.,  3.,  6., 12.,  0.,  0.,  8., 13., 13.,
          

In [15]:
print(w_tile.size())
print(a_tile.size())
print(out.size())

torch.Size([1, 8, 8, 9])
torch.Size([1, 8, 36])
torch.Size([8, 16])
